In [1]:
import os, sys, glob
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import pandas as pd

SAMPLES_DIR = 'artifacts/samples/EVAL-targetgen_noise-lw11111-len200-beta0.02-steps1000-epsilon-TransferFalse-ssfrac1-L12-C768-cfdg_ddpm-w=1.2-p=0.1-k=3-dia=2-4'
BM_SAMPLES_DIR = 'artifacts/samples/EVAL-basismixer'
SP_SAMPLES_DIR = 'artifacts/samples/EVAL-scoreperformer'

In [2]:
dexter_eval_dir = glob.glob(f"{SAMPLES_DIR}/**/*_distribution.csv", recursive=True)
bm_eval_dir = glob.glob(f"{BM_SAMPLES_DIR}/**/*_distribution.csv", recursive=True)
sp_eval_dir = glob.glob(f"{SP_SAMPLES_DIR}/**/*_distribution.csv", recursive=True)

In [27]:
def replace_value(table):
    table = table.replace([np.inf, -np.inf], np.nan)
    table = table.replace(["--"], np.nan)
    for value in table.iloc[1]:
        if float(value) < 0:
            table.iloc[1] = table.iloc[1].replace(value, np.nan) # replace dummy value in KL divergence
    return table

def get_table_mean(eval_dir, N=10000, std=False):
    all_eval_table = [replace_value(pd.read_csv(eval_csv_path)) for eval_csv_path in eval_dir][:N] # mean ignores nan but not ignores inf. So replace inf with nan
    all_eval_table = pd.concat(all_eval_table)
    if std:
        eval_table_mean = all_eval_table.groupby(level=0).std()
    else:
        eval_table_mean = all_eval_table.groupby(level=0).mean()
    eval_table_mean = eval_table_mean.set_index(pd.Index(["Deviation", "KL Divergence", "Correlation"]))

    return eval_table_mean


In [47]:
dexter_pred_feats = 'artifacts/samples/EVAL-targetgen_noise-lw11111-len200-beta0.02-steps1000-epsilon-TransferFalse-ssfrac1-L12-C768-cfdg_ddpm-w=1.2-p=0.1-k=3-dia=2-4/epoch=0/batch=0/4_Mozart_K331_1st-mov_feats_pred.csv'
dexter_label_feats = 'artifacts/samples/EVAL-targetgen_noise-lw11111-len200-beta0.02-steps1000-epsilon-TransferFalse-ssfrac1-L12-C768-cfdg_ddpm-w=1.2-p=0.1-k=3-dia=2-4/epoch=0/batch=0/4_Mozart_K331_1st-mov_feats_label.csv'

dexter_pred_feats = pd.read_csv(dexter_pred_feats)
dexter_label_feats = pd.read_csv(dexter_label_feats)

((dexter_pred_feats['asynchrony_feature.pitch_cor'] - dexter_label_feats['asynchrony_feature.pitch_cor']) / dexter_label_feats['asynchrony_feature.pitch_cor']).to_list()

[-0.8356156258767886,
 -0.8356156258767886,
 -0.8356156258767886,
 -4.549260595336291,
 -4.549260595336291,
 -4.549260595336291,
 -4.549260595336291,
 -4.549260595336291,
 0.03598625452298308,
 0.03598625452298308,
 0.03598625452298308,
 0.03598625452298308,
 0.0,
 0.0,
 nan,
 nan,
 0.0,
 0.0,
 0.0,
 0.0,
 nan,
 0.0,
 0.0,
 -3.1276222996460357,
 -3.1276222996460357,
 -3.1276222996460357,
 -3.1276222996460357,
 nan,
 nan,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -2.1552110407716065,
 -2.1552110407716065,
 -2.1552110407716065,
 nan,
 nan,
 0.012238449019150718,
 0.012238449019150718,
 0.012238449019150718,
 nan,
 nan,
 -0.44725220277541483,
 -0.44725220277541483,
 -0.44725220277541483,
 nan,
 -0.3691590801200631,
 -0.3691590801200631,
 -0.3691590801200631,
 -0.009375043919728364,
 -0.009375043919728364,
 -0.009375043919728364,
 -0.009375043919728364,
 0.0,
 0.0,
 0.09306211123139023,
 0.09306211123139023,
 0.09306211123139023,
 -2.0,
 -2.0,
 -0.04907072157768789,
 -0.049070721577

In [48]:
get_table_mean(dexter_eval_dir)


,articulation_feature.kor,asynchrony_feature.pitch_cor,asynchrony_feature.vel_cor,asynchrony_feature.delta,dynamics_feature.agreement,dynamics_feature.consistency_std,dynamics_feature.ramp_cor,dynamics_feature.tempo_cor,pedal_feature.onset_value,tempo_curve,vel_curve
Deviation,-25.446637,-1.379293e+13,7.169896e+12,2.802631,-0.639896,0.578770,-1.386965,-0.238807,1.472649,0.271540,0.250374
KL Divergence,1.338049,1.521246e-01,1.465462e-01,0.600663,0.018718,0.088446,0.319644,2.126103,1.321059,2.192403,1.688907
Correlation,0.176140,6.905795e-02,5.008594e-02,0.293237,0.359625,0.916228,0.200262,0.034027,0.019033,0.091459,0.324206


In [33]:
get_table_mean(bm_eval_dir)


,articulation_feature.kor,asynchrony_feature.pitch_cor,asynchrony_feature.vel_cor,asynchrony_feature.delta,dynamics_feature.agreement,dynamics_feature.consistency_std,dynamics_feature.ramp_cor,dynamics_feature.tempo_cor,pedal_feature.onset_value,tempo_curve,vel_curve
Deviation,-195.938394,-0.802881,-0.981026,2.801515,-0.835618,0.116320,-1.126905,-3.785589,-1.0,0.325488,0.310339
KL Divergence,0.143257,0.216394,0.116405,1.059458,0.020820,0.080058,2.069532,6.768318,NaN,0.775446,0.609890
Correlation,0.054497,0.190260,0.010090,0.175163,0.164363,0.894938,-0.107907,-0.008320,NaN,0.056843,0.210751


In [34]:
get_table_mean(sp_eval_dir)

,articulation_feature.kor,asynchrony_feature.pitch_cor,asynchrony_feature.delta,dynamics_feature.agreement,dynamics_feature.consistency_std,dynamics_feature.ramp_cor,dynamics_feature.tempo_cor,pedal_feature.onset_value,tempo_curve,vel_curve
Deviation,-1203.116816,0.006036,12.392316,-0.910059,2.123997,-0.568833,0.273695,-1.0,8.439348,0.033967
KL Divergence,0.435141,0.036456,3.463461,0.031530,0.108558,0.170209,1.349811,NaN,0.254056,0.415702
Correlation,0.062876,0.657484,0.275984,0.090684,0.926963,0.152800,0.038515,NaN,0.002326,0.050970
